# Kaggle Competition on Mashtots Dataset

In this homework, you do not need to implement any algorithm from scratch, instead you will try to create an algorithm in order to classify Armenian handwritten characters. Do the following steps in order to get started:

* Visit [the page of the competition]https://www.kaggle.com/c/mashtots-dataset-v2/overview) and press "Join Competition"

* You will need to sign in to kaggle if you have an account already and if you don't, you will need to create one

* After signing in you will need to accept the rules of the competition by pressing "I understand and accept"

* Go to the "Data" tab, read about the dataset and then download the dataset files

* Upload the dataset into Google Drive, so that you can access it from Colab

* At this point you should be able to start your experiments.

# Mounting Google Drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/ML/MashtotsDataset

/content/drive/MyDrive/ML/MashtotsDataset


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nickkharin","key":"fc32b98e0e9114f5a6026f36b35a1aff"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c mashtots-dataset-v2

 99% 218M/219M [00:10<00:00, 23.5MB/s]
100% 219M/219M [00:10<00:00, 21.4MB/s]


In [ ]:
!unzip mashtots-dataset-v2.zip

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid') # Plot style
plt.rcParams['figure.figsize'] = (12.0, 8.0)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

<ipython-input-5-d351138422ef>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid') # Plot style


## Data Processing

If you have noticed, the training data consists of raw images, which means that we need to process those images and construct a dataset (pd.DataFrame or np.array) where each row will be an image vector and the last column will be its corresponding label. Complete the missing parts of the code snippet below in order to construct the dataset.

In [ ]:
root_dir = "Train/Train"

# getting the list of folder names (label encodings)
sub_folders = os.listdir(root_dir)

In [ ]:
# empty list for the data
train = []

# going over the folders
count = 0
total_images = sum([len(files) for r, d, files in os.walk(root_dir)])

for sub_folder in sub_folders:
    images = os.listdir(os.path.join(root_dir, sub_folder))
    for image_name in images:
        image_path = os.path.join(root_dir, sub_folder, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is not None:
            if image.shape[0] != 64 or image.shape[1] != 64:
                image = cv2.resize(image, (64, 64))

            feature_vector = image.flatten()
            train.append([*feature_vector, int(sub_folder)])
        else:
            print(f"Не удалось загрузить изображение: {image_path}")

        count += 1
        if count % 100 == 0:
            print(f"Обработано изображений: {count} из {total_images}")

print(f"Всего обработано изображений: {count}")

In [ ]:
train = pd.DataFrame(train)

Have a look the resulting data data

In [ ]:
train.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_4087,pixel_4088,pixel_4089,pixel_4090,pixel_4091,pixel_4092,pixel_4093,pixel_4094,pixel_4095,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


It's a good idea to save the processed dataset in your Drive, so that you don't need go through the above process every time.

In [ ]:
train.to_csv('train.csv', index=False)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ML/MashtotsDataset/train.csv')

In [ ]:
feature_columns = [f'pixel_{i}' for i in range(4096)]  # 4096 = 64x64
train.columns = feature_columns + ['label']

## Constructing a Validation set

Since you do not have a separate labeled test data for validation, you should construct (put aside) your own validation set from the whole training data, in order to choose good models.
You are free to decide how much data you will use for what purposes, but remember to use data wisely.

In [ ]:
from sklearn.model_selection import train_test_split

X = train[feature_columns]
y = train['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train = X_train.reshape(-1, 64, 64, 1)
X_val = X_val.reshape(-1, 64, 64, 1)

X_train = X_train / 255.0
X_val = X_val / 255.0

## Modelling

Perform experiments with different algorithms that we have learned and choose a good algorithm for final evaluation.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(sub_folders), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
1752/1752 [==============================] - 20s 8ms/step - loss: 1.2525 - accuracy: 0.6692 - val_loss: 0.4981 - val_accuracy: 0.8526
Epoch 2/20
1752/1752 [==============================] - 13s 8ms/step - loss: 0.4458 - accuracy: 0.8685 - val_loss: 0.3269 - val_accuracy: 0.9009
Epoch 3/20
1752/1752 [==============================] - 13s 7ms/step - loss: 0.3525 - accuracy: 0.8936 - val_loss: 0.3202 - val_accuracy: 0.9022
Epoch 4/20
1752/1752 [==============================] - 13s 7ms/step - loss: 0.3022 - accuracy: 0.9086 - val_loss: 2.2488 - val_accuracy: 0.5380
Epoch 5/20
1752/1752 [==============================] - 13s 8ms/step - loss: 0.2712 - accuracy: 0.9168 - val_loss: 0.2420 - val_accuracy: 0.9266
Epoch 6/20
1752/1752 [==============================] - 13s 7ms/step - loss: 0.2362 - accuracy: 0.9282 - val_loss: 0.2375 - val_accuracy: 0.9294
Epoch 7/20
1752/1752 [==============================] - 13s 7ms/step - loss: 0.2116 - accuracy: 0.9357 - val_loss: 0.6469 - val_ac

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(GlobalAveragePooling2D())

model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(sub_folders), activation='softmax', kernel_regularizer=l2(0.001)))  # len(sub_folders) - количество классов

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_68 (Conv2D)          (None, 64, 64, 32)        320       
                                                                 
 batch_normalization_18 (Ba  (None, 64, 64, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_69 (Conv2D)          (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_34 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 32, 32, 32)        0         
                                                                 
 conv2d_70 (Conv2D)          (None, 32, 32, 64)       

In [ ]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
1752/1752 [==============================] - 22s 10ms/step - loss: 1.6051 - accuracy: 0.6505 - val_loss: 0.7880 - val_accuracy: 0.8358
Epoch 2/20
1752/1752 [==============================] - 16s 9ms/step - loss: 0.6371 - accuracy: 0.8736 - val_loss: 0.5833 - val_accuracy: 0.8818
Epoch 3/20
1752/1752 [==============================] - 16s 9ms/step - loss: 0.5168 - accuracy: 0.8992 - val_loss: 0.4241 - val_accuracy: 0.9256
Epoch 4/20
1752/1752 [==============================] - 16s 9ms/step - loss: 0.4602 - accuracy: 0.9114 - val_loss: 2.3359 - val_accuracy: 0.6557
Epoch 5/20
1752/1752 [==============================] - 16s 9ms/step - loss: 0.4122 - accuracy: 0.9223 - val_loss: 2.1364 - val_accuracy: 0.5937
Epoch 6/20
1752/1752 [==============================] - 16s 9ms/step - loss: 0.3789 - accuracy: 0.9293 - val_loss: 0.3598 - val_accuracy: 0.9342
Epoch 7/20
1752/1752 [==============================] - 16s 9ms/step - loss: 0.3538 - accuracy: 0.9365 - val_loss: 0.3528 - val_a

In [ ]:
test_loss, test_accuracy = model.evaluate(X_val, y_val)
print('Точность на валидационных данных:', test_accuracy)

438/438 [==============================] - 1s 3ms/step - loss: 0.3149 - accuracy: 0.9528
Точность на валидационных данных: 0.9527547955513


## Generating predictions

At this stage you should have good performing model that you want to test on kaggle.

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/ML/MashtotsDataset/new_test/new_test.csv')
test.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,0,0,4,0,0,0,0,0,0,0,...,0,0,3,0,0,2,0,1,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test = test.values

X_test = X_test.reshape(-1, 64, 64, 1)

X_test = X_test / 255.0

In [ ]:
predictions = model.predict(X_test)
test_loss, test_accuracy = model.evaluate(X_val, y_val)
predicted_classes = np.argmax(predictions, axis=1)
submission_df = pd.DataFrame({'Id': np.arange(1, X_test.shape[0]+1),
                              'Category': predicted_classes})
submission_df.to_csv('/content/drive/MyDrive/ML/MashtotsDataset/my_submission6.csv', index=False)

438/438 [==============================] - 1s 3ms/step - loss: 0.6518 - accuracy: 0.8646


* Download the generated ``my_submission.csv`` file and upload it to kaggle by pressing "Submit predictions" and following the steps.

* You have to describe your submission under the "Describe Submission" section with some details about the algorithm/hyperparameters that you used

* Take a screenshot of the kaggle web page after each submission, where one can see the submission file name, its description and the achieved accuracy score.

* **You will need to send those screenshots along with this notebook**

* After the submission you will see the accuracy of your model and your position in the public leaderboard

* You can only submit 5 files per day

* You can not participate in teams (for this homework)

# Motivation

* You need to make at least 5 submissions with accuracy **above 40%** in order to get full grade from this homework

* Those of you who get over 40% will get the corresponding grade

* The person who gets the highest score among all participants will get 100%.